In [2]:
import os

# Define the target directory
target_directory = os.getcwd()  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop\azure-ai-agent-services-demo


## SharePoint Integration with Azure AI Agents

SharePoint can be used as a data source to ground Azure AI Agents with documents stored securely. When a user queries, Azure AI Agents determine if SharePoint should be used. If so, the query is sent via the SharePoint tool, which checks the user's M365 copilot license and retrieves relevant documents based on the user's identity and access. The scope includes all supported documents in the SharePoint site, and responses are generated based on the retrieved information.

### Prerequisites
- Existing SharePoint site with developer access
- M365 copilot license for developers and end users
- AOAI resource and AI project in regions: westus, japaneast, francecentral

### RABC Roles
- AI Developer role to CRUD a SharePoint tool in Azure AI Agent
- AI Developer role for end users for OBO authentication

### Supported Capabilities
- Grounding with all supported documents in a SharePoint site
- Supported document types: PDF, Word, PPT, txt, .aspx (text data only)
- Automatic indexing of updated documents
- Responses based on end user’s document access
- Connection to a maximum of 1 SharePoint site

### Known Limitations
- Retrieval and grounding quality issues, especially with complex formatting, tables, columns, and images
- Text-sparse file types (e.g., PPT) may return poorer results than text-rich types (e.g., docx)
- Grounding with specific libraries or multiple sites is not supported

In [ ]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
FILE: sample_agents_sharepoint.py
DESCRIPTION:
    This sample demonstrates how to use agent operations with the 
    Sharepoint tool from the Azure AI Agents service using a synchronous client.
    The sharepoint tool is currently available only to whitelisted customers.
    For access and onboarding instructions, please contact azureagents-preview@microsoft.com.
glharper marked this conversation as resolved.
USAGE:
    python sample_agents_sharepoint.py
    Before running the sample:
    pip install azure-ai-projects azure-identity python-dotenv
    Set this environment variables with your own values:
    PROJECT_CONNECTION_STRING - the Azure AI Project connection string, as found in your AI Studio Project.
"""

import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import SharepointTool
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Create an Azure AI Client from a connection string, copied from your AI Studio project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# Initialize Sharepoint tool with connection id
sharepoint_connection = project_client.connections.get(
    connection_name=os.environ["CONNECTION_NAME"],
)
conn_id = sharepoint_connection.id
print(conn_id)

sharepoint = SharepointTool(connection_id=conn_id)

# Create agent with Sharepoint tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_NAME"],
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=sharepoint.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hello, summarize the key points of the <sharepoint_resource_document>",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")